In [4]:
@show Meta.parse("x+y")
@show Meta.parse("x+y")|>dump

Meta.parse("x+y") = :(x + y)
Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol x
    3: Symbol y
Meta.parse("x+y") |> dump = nothing


In [5]:
@show Meta.parse("x")|> dump

Symbol x
Meta.parse("x") |> dump = nothing


In [14]:
@show Meta.parse("""print("x") """)|>dump

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol print
    2: String "x"
Meta.parse("print(\"x\") ") |> dump = nothing


In [24]:
@show Meta.parse("""open("hoge.txt",read=true,write=true)""")|>dump

Expr
  head: Symbol call
  args: Array{Any}((4,))
    1: Symbol open
    2: String "hoge.txt"
    3: Expr
      head: Symbol kw
      args: Array{Any}((2,))
        1: Symbol read
        2: Bool true
    4: Expr
      head: Symbol kw
      args: Array{Any}((2,))
        1: Symbol write
        2: Bool true
Meta.parse("open(\"hoge.txt\",read=true,write=true)") |> dump = nothing


# constructing expression objects manually

In [25]:
Expr(:call,:+,:x,:y)

:(x + y)

In [27]:
:(x=1+1)|>dump

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol x
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 1
        3: Int64 1


In [29]:
:(begin println("hello world")
    pirnln("hello world!!!!")
    end)|>dump


Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 1
      file: Symbol In[29]
    2: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol println
        2: String "hello world"
    3: LineNumberNode
      line: Int64 2
      file: Symbol In[29]
    4: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol pirnln
        2: String "hello world!!!!"


In [31]:
:(if 2>1 "good" 
    else 
        "bad"
    end)|>dump

Expr
  head: Symbol if
  args: Array{Any}((3,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol >
        2: Int64 2
        3: Int64 1
    2: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: LineNumberNode
          line: Int64 1
          file: Symbol In[31]
        2: String "good"
    3: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: LineNumberNode
          line: Int64 3
          file: Symbol In[31]
        2: String "bad"


In [32]:
macro hello()
    return :(for i in 1:3
        println("hellow world")
        end
    )
end

@hello

hellow world
hellow world
hellow world


In [34]:
macro hello(n)
    return:(for i in 1:$n
            println("hello world")
        end
        )
end
@hello(10)

hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world


In [38]:
@macroexpand @hello 4

:(for var"#23#i" = 1:4
      #= In[34]:3 =#
      Main.println("hello world")
  end)

In [40]:
@macroexpand @show x=1

quote
    Base.println("x = 1 = ", Base.repr(begin
                #= show.jl:641 =#
                var"#25#value" = (x = 1)
            end))
    var"#25#value"
end

In [42]:
macro identity(ex)
    dump(ex)
    return ex
end

function foo()
    return @identity 1+2+3
end

Expr
  head: Symbol call
  args: Array{Any}((4,))
    1: Symbol +
    2: Int64 1
    3: Int64 2
    4: Int64 3


foo (generic function with 1 method)

In [48]:
@identity 123
x=123


Int64 123


123

In [53]:
macro squared(ex)
    return :($(ex)*$(ex))
end

@squared 123


15129

In [56]:
function foo()
    x=2
    return @squared x
end
foo()

@code_lowered foo()

CodeInfo(
1 ─      x = 2
│   %2 = Main.x * Main.x
└──      return %2
)

In [60]:
macro squared(ex)
    return :($(esc(ex))*$(esc(ex)))
end
function foo()
    x=2
    return @squared x
end
x=23
@show @squared x
foo()

#= In[60]:9 =# @squared(x) = 529


4

In [63]:
:(sin(x))|>dump
:(sin(sin(x)))|>dump

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol sin
    2: Symbol x
Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol sin
    2: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol sin
        2: Symbol x


In [65]:
macro compose_twice(ex)
    @assert ex.head==:call
    @assert length(ex.args)==2
    me=copy(ex)
    ex.args[2]=me
    return ex
end



@compose_twice (macro with 1 method)

In [73]:
@compose_twice(sin(1))==sin(sin(1))

true

In [6]:
macro ntimes(n,ex)
    quote 
        times=$(esc(n))
        for i in 1:times
            $(esc(ex))
        end            
    end
end


function foo()
    times=0
    @ntimes 4 println("helloworlld")
    printlnt("times=",times)
end


foo (generic function with 1 method)

In [9]:
function for_loop()
    for i in 1:10
        print("hello:$i")
    end
end

@code_lowered for_loop()

CodeInfo(
1 ─ %1  = 1:10
│         @_2 = Base.iterate(%1)
│   %3  = @_2 === nothing
│   %4  = Base.not_int(%3)
└──       goto #4 if not %4
2 ┄ %6  = @_2
│         i = Core.getfield(%6, 1)
│   %8  = Core.getfield(%6, 2)
│   %9  = Base.string("hello:", i)
│         Main.print(%9)
│         @_2 = Base.iterate(%1, %8)
│   %12 = @_2 === nothing
│   %13 = Base.not_int(%12)
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return
)

In [14]:
using Unrolled
@unroll function for_loop2(x)
    @unroll for i in x
        prinln("hello:$i")
    end
end
@code_lowered for_loop2(1:10)

CodeInfo(
    @ C:\Users\tarob\.julia\packages\Unrolled\26uDc\src\Unrolled.jl:127 within `for_loop2'
   ┌ @ C:\Users\tarob\.julia\packages\Unrolled\26uDc\src\Unrolled.jl:127 within `macro expansion' @ In[14]:3
1 ─│ %1  = x
│  │       @_3 = Base.iterate(%1)
│  │ %3  = @_3 === nothing
│  │ %4  = Base.not_int(%3)
└──│       goto #4 if not %4
2 ┄│ %6  = @_3
│  │       i = Core.getfield(%6, 1)
│  │ %8  = Core.getfield(%6, 2)
│  │ @ C:\Users\tarob\.julia\packages\Unrolled\26uDc\src\Unrolled.jl:127 within `macro expansion' @ In[14]:3 @ C:\Users\tarob\.julia\packages\Unrolled\26uDc\src\Unrolled.jl:110
│  │ %9  = Base.string("hello:", i)
│  │       Main.prinln(%9)
│  │ @ C:\Users\tarob\.julia\packages\Unrolled\26uDc\src\Unrolled.jl:127 within `macro expansion' @ In[14]:3
│  │       @_3 = Base.iterate(%1, %8)
│  │ %12 = @_3 === nothing
│  │ %13 = Base.not_int(%12)
└──│       goto #4 if not %13
3 ─│       goto #2
   │ @ C:\Users\tarob\.julia\packages\Unrolled\26uDc\src\Unrolled.jl:127 within `mac

In [27]:
:(begin
        x=12
        if typeof(x)==Integer
            preintln("hoge")
        elseif x<0
            println("x is negative")
        else
            print("aaaaaaaaa")
        end
    
    end) |> dump


Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[27]
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol x
        2: Int64 12
    3: LineNumberNode
      line: Int64 3
      file: Symbol In[27]
    4: Expr
      head: Symbol if
      args: Array{Any}((3,))
        1: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol ==
            2: Expr
              head: Symbol call
              args: Array{Any}((2,))
                1: Symbol typeof
                2: Symbol x
            3: Symbol Integer
        2: Expr
          head: Symbol block
          args: Array{Any}((2,))
            1: LineNumberNode
              line: Int64 4
              file: Symbol In[27]
            2: Expr
              head: Symbol call
              args: Array{Any}((2,))
                1: Symbol preintln
                2: String "hoge"
        3: Expr
          head: Symbol 

In [29]:
function foo()
    y=1
end
foo()
y

LoadError: UndefVarError: y not defined

In [32]:
function foo2()
    eval(:(y=1))
end

foo2()
#evaluation of the expression happens in the global scope not local
@show y

y = 1


1

In [38]:
macro hello_1(n)
    print("n=$n")
    return :(for i in 1:$n
        println("hello world")
        end)
end
@hello_1(10)
n

n=10hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world
hello world


LoadError: UndefVarError: n not defined

In [40]:
a=1
b="hoge"
c=:hello
macro showme(x)
    @show x
end4
    
@showme(a)
@showme(b)
@showme(c)

LoadError: syntax: incomplete: "macro" at In[40]:4 requires end

In [42]:
:(for i in 1:3
    println("hello world")
    end)

:(for i = 1:10
      #= In[42]:2 =#
      println(i)
  end)

In [60]:
macro hello(n)
    return :(for i in 1:$n
        println("hello world $i")
        end
        )
end

@macroexpand @hello(3)


:(for var"#43#i" = 1:3
      #= In[60]:3 =#
      Main.println("hello world $(var"#43#i")")
  end)

In [61]:
function test(y23)
    @hello y23
end
test(34)

LoadError: UndefVarError: y23 not defined

In [53]:
@hello(3)

hello world 1
hello world 2
hello world 3


In [55]:
keval(:(x=m))


LoadError: UndefVarError: m not defined